## Feature Analysis
---

In [5]:
import sys, os, json
sys.path.insert(1, "../..")

import pickle as pkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
fd_dev1_v1 = pd.read_csv("./artifacts/dev1-v1-model-artifacts/feature_dict.csv", index_col=0)
fd_dev2_v1 = pd.read_csv("./artifacts/dev2-v1-model-artifacts/feature_dict.csv", index_col=0)
fd_dev2_v2 = pd.read_csv("./artifacts/dev2-v2-model-artifacts/feature_dict.csv", index_col=0)

In [5]:
fd = pd.concat([fd_dev1_v1,
                fd_dev2_v1,
                fd_dev2_v2], axis=0)
fd.shape

(96, 6)

In [7]:
# freq of freq table
fd.description.value_counts().value_counts()

1    19
2     8
3     5
Name: description, dtype: int64

In [9]:
fd.attr_grp.value_counts()

Balance Amount Payment               15
Age/Recency                          13
Payment variance                     13
Public Records/Inquiry               10
Payment magnitude                     9
Balance variance                      7
Available credit amount magnitude     7
Ratios/Percentage                     6
Balance to limit ratio variance       4
Rank                                  4
Trade Count                           2
Balance over limit activity           2
Slope                                 2
Credit amount variance                1
Quarterly average                     1
Name: attr_grp, dtype: int64

In [31]:
fd_dev1_v1["rank"] = range(1, len(fd_dev1_v1)+1)
fd_dev2_v1["rank"] = range(1, len(fd_dev2_v1)+1)
fd_dev2_v2["rank"] = range(1, len(fd_dev2_v2)+1)

In [32]:
fd_dev1_v1.head()

,description,long description,attr_grp,categorical,type,dir,rank
field_name,,,,,,,
p13_iln5820,ILN5820,Total monthly payment on open installment trad...,Balance Amount Payment,"[999999999, 999999998, 999999997, 999999996]",int,1,1
p13_alj8120,ALJ8120,Number of months since the most recently opene...,Age/Recency,"[9999, 9998, 9996, 9994]",int,1,2
p13_iqf9540,IQF9540,Number of days since the most recent personal ...,Public Records/Inquiry,[9999],int,-1,3
p13_all5321,ALL5321,Total credit amount on open trades,Balance Amount Payment,"[999999999, 999999996]",int,-1,4
t11_taxm1203,NaN,Total difference between month 0 (adjusted) ba...,Balance variance,"[999999999, 999999998, 999999997, 999999996]",int,1,5


In [39]:
fts = fd.index.unique()

In [48]:
fd_present = pd.DataFrame(index=fts)
fd_present = pd.merge(fd_present, fd_dev1_v1["rank"].rename("dev1_v1"), left_on="field_name", right_index=True, how="left")
fd_present = pd.merge(fd_present, fd_dev2_v1["rank"].rename("dev2_v1"), left_on="field_name", right_index=True, how="left")
fd_present = pd.merge(fd_present, fd_dev2_v2["rank"].rename("dev2_v2"), left_on="field_name", right_index=True, how="left")
fd_present = pd.merge(fd_present, fd["long description"].drop_duplicates(), left_on="field_name", right_index=True, how="left")


In [50]:
fd_present["mean"] = fd_present[["dev1_v1", "dev2_v1", "dev2_v2"]].mean(axis=1)
fd_present.sort_values("mean", inplace=True)
fd_present.to_csv("./artifacts/models/benchmark-top-features.csv")

In [52]:
fd_present.shape

(56, 5)

In [26]:
from src.utils.governance import get_feature_by_importance

with open("config.json", "r") as f:
    config = json.load(f)

exp_dict = pd.read_csv(config["meta"]["exp_dict_path"])

with open("./artifacts/models/lightgbm-combined_v1_benchmark_mc.pkl", "rb") as f:
    lgbm_v1 = pkl.load(f)
    
with open("./artifacts/models/lightgbm-combined_v2_benchmark_mc.pkl", "rb") as f:
    lgbm_v2 = pkl.load(f)
    
with open("./artifacts/models/uw_combined_target1_lgbm_mc_benchmark.pkl", "rb") as f:
    cs_v1 = pkl.load(f)
    
with open("./artifacts/models/uw_combined_target2_lgbm_mc_benchmark.pkl", "rb") as f:
    cs_v2 = pkl.load(f)
    
    
feature_by_imp_v1 = get_feature_by_importance(lgbm_v1)
feature_by_imp_v2 = get_feature_by_importance(lgbm_v2)
feature_by_imp_cs_v1 = get_feature_by_importance(cs_v1)
feature_by_imp_cs_v2 = get_feature_by_importance(cs_v2)

Trying to unpickle estimator LabelEncoder from version 0.24.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [55]:
def get_feature_descriptions(features, data_dict, lgbm):
    dd_cols = ["field_name", "description", "long description", "attr_grp", "categorical", "type"]
    dd = data_dict[dd_cols]
    dd = dd[dd.field_name.isin(lgbm.feature_name_)]
    dd.set_index("field_name", inplace=True)
    
    dd_fields = set(data_dict.field_name.unique())
    bureau_fts = [f for f in features if f in dd_fields]
    bureau_dd = dd.loc[bureau_fts]
    
    if len(lgbm.feature_name_) == len(bureau_fts):
        return bureau_dd
    
    custom_ft = [f for f in features if f not in dd_fields]
    custom_dd = pd.DataFrame(index=custom_ft)
    return pd.concat([bureau_dd, custom_dd]).loc[features]

In [56]:
feature_dict_v1 = get_feature_descriptions(feature_by_imp_v1, exp_dict, lgbm_v1)
dir_map = dict(zip(lgbm_v1.feature_name_, lgbm_v1.get_params()["monotone_constraints"]))
feature_dict_v1["dir"] = [dir_map[k] for k in feature_dict_v1.index.values]
feature_dict_v1["rank"] = range(1, len(feature_dict_v1)+1)

feature_dict_v2 = get_feature_descriptions(feature_by_imp_v2, exp_dict, lgbm_v2)
dir_map = dict(zip(lgbm_v2.feature_name_, lgbm_v2.get_params()["monotone_constraints"]))
feature_dict_v2["dir"] = [dir_map[k] for k in feature_dict_v2.index.values]
feature_dict_v2["rank"] = range(1, len(feature_dict_v2)+1)

cs_feature_dict_v1 = get_feature_descriptions(feature_by_imp_cs_v1, exp_dict, cs_v1)
dir_map = dict(zip(cs_v1.feature_name_, cs_v1.get_params()["monotone_constraints"]))
cs_feature_dict_v1["dir"] = [dir_map[k] for k in cs_feature_dict_v1.index.values]
cs_feature_dict_v1["rank"] = range(1, len(cs_feature_dict_v1)+1)

cs_feature_dict_v2 = get_feature_descriptions(feature_by_imp_cs_v2, exp_dict, cs_v2)
dir_map = dict(zip(cs_v2.feature_name_, cs_v2.get_params()["monotone_constraints"]))
cs_feature_dict_v2["dir"] = [dir_map[k] for k in cs_feature_dict_v2.index.values]
cs_feature_dict_v2["rank"] = range(1, len(cs_feature_dict_v2)+1)

In [92]:
feature_dict_v1.head()

,description,long description,attr_grp,categorical,type,dir,rank
field_name,,,,,,,
p13_iqf9540,IQF9540,Number of days since the most recent personal ...,Public Records/Inquiry,[9999],int,-1,1
p13_alj8120,ALJ8120,Number of months since the most recently opene...,Age/Recency,"[9999, 9998, 9996, 9994]",int,1,2
t11_tmti2752,NaN,Total actual payment over the last 6 months on...,Payment magnitude,"[999999999, 999999998, 999999997, 999999996]",int,-1,3
p13_pil8132,PIL8132,Number of days since the most recently opened ...,Age/Recency,"[99999, 99998, 99996, 99994]",int,-1,4
t11_trev0722,NaN,Difference between Q1 (m00-m03) total average ...,Payment variance,"[999999999, 999999998, 999999997, 999999996]",int,1,5


In [59]:
fd = pd.concat([feature_dict_v1,
                feature_dict_v2,
                cs_feature_dict_v1,
                cs_feature_dict_v2
                ], axis=0)
fd.shape

(128, 7)

In [99]:
fts = fd.index.unique()
data_dict = exp_dict[["field_name",
                      "long description", "attr_grp", 
                      "categorical", "type"]]

In [100]:
fd_present = pd.DataFrame(index=fts)
fd_present = pd.merge(fd_present, feature_dict_v1["rank"].rename("prescreen_combined_v1"), left_index=True, right_index=True, how="left")
fd_present = pd.merge(fd_present, feature_dict_v2["rank"].rename("prescreen_combined_v2"), left_index=True, right_index=True, how="left")
fd_present = pd.merge(fd_present, cs_feature_dict_v1["rank"].rename("credit_scoring_combined_v1"), left_index=True, right_index=True, how="left")
fd_present = pd.merge(fd_present, cs_feature_dict_v2["rank"].rename("credit_scoring_combined_v2"), left_index=True, right_index=True, how="left")
fd_present = pd.merge(fd_present, data_dict, left_index=True, right_on="field_name")

In [103]:
cols = ["prescreen_combined_v1", "prescreen_combined_v2",
        "credit_scoring_combined_v1", "credit_scoring_combined_v2"]
fd_present["-freq"] = -(~fd_present.isna()).sum(axis=1)
fd_present["mean"] = fd_present[cols].mean(axis=1)
fd_present["min"] = fd_present[cols].min(axis=1)
fd_present.sort_values(["-freq", "min"], inplace=True)

cols = ["field_name", "prescreen_combined_v1", "prescreen_combined_v2",
        "credit_scoring_combined_v1", "credit_scoring_combined_v2", "long description", "attr_grp", 
        "categorical", "type"]
fd_present[cols].to_csv("./artifacts/models/benchmark-top-features-combined.csv")